### Imports

In [ ]:
import os
import time

import numpy as np
import pandas as pd
from pathlib import Path

import string
import itertools

import sys
sys.path.append("..")

from simulation.simulation_utils import simulate
from simulation.simulation_tools import get_optimal_sim_XY

rng = np.random.default_rng()

COL_NAMES = list(string.ascii_uppercase) + ["".join(a) for a in list(itertools.permutations(list(string.ascii_uppercase), r=2))]

### Single Simulation

In [ ]:
# data
data_path = list(Path(".").resolve().parents)[1] / 'data' / 'cp_style' / 'increasing_edges_cp_1' / 'data'
fn = os.listdir(data_path)[0]
true_data = pd.read_csv(data_path / fn)

# TCS configuration
cfg = {
    "cd_method" : "PCMCI", 
    "cd_kwargs" : None, 
    "pred_method" : "GBR", 
    "pred_kwargs" : None, 
    "o_approximation" : "spline", 
    "noise_approximation" : "spline",
}

# simulate
start_time = time.time()
sim_data, sim_scm, funcs_and_noise, scores = simulate(
    true_data=true_data, 
    true_label=None, 
    n_samples=500, 
    verbose=True, 
    **cfg
)
elapsed_time = time.time() - start_time
print(f"LOG : Single Simulation : Elapsed time for single simulation: {round(elapsed_time, 2)}")

### Optimized Simulation

In [ ]:
# data
data_path = list(Path(".").resolve().parents)[1] / 'data' / 'fMRI' / 'timeseries'
data_path = list(Path(".").resolve().parents)[1] / 'data' / 'MvTS' / 'ETTh1'
data_path = list(Path(".").resolve().parents)[1] / 'data' / 'cp_style' / 'cp_0_NL_3L' / 'data'
fn = os.listdir(data_path)[8]
true_data = pd.read_csv(data_path / fn)

# simulate
start_time = time.time()
res = get_optimal_sim_XY(
        true_data = true_data, 
        CONFIGS = None, 
        optimal_det_config = None,
        optimal_det_func = None, 
        sparsity_penalty=True,
        bias_correction=True,
        verbose = True
)

elapsed_time = time.time() - start_time
print(f"LOG : Optimized Simulation : Elapsed time for optimized simulation: {round(elapsed_time, 2)}")

### Single Simulation - BBC

In [ ]:
from utils import ts_to_lagged

""" ____________ Dummy Data on Triginometric Functions ____________ """
index = np.linspace(0, 500, 502)
col_1 = np.sin(index) + np.random.rand(len(index))*0.2 
col_2 = []
for i, x1 in zip(index[1:], col_1[1:]):
    col_2.append(1 + np.cos(i) + np.cos(x1)*0.4 + 0.5 + np.random.rand()*0.2)
col_3 = []
for i, x1, x2 in zip(index[2:], col_1[2:], col_2[1:]):
    col_3.append(2 + np.tanh(i) + np.sin(x1) + np.cos(x2)*0.3 + 0.5 + np.random.rand()*0.2)
index = np.array(index[2:])
col_1 = np.array(col_1[2:])
col_2 = np.array(col_2[1:])
col_3 = np.array(col_3[:])

# true
data = np.stack([col_1, 
                 col_2, 
                 col_3], axis=1)
true_data = pd.DataFrame(data=data, columns=COL_NAMES[:data.shape[1]])

# simulated
data = np.stack([col_1 + np.random.normal(loc=0.5, scale=0.5, size=len(col_1)), 
                 col_2 + np.random.normal(loc=0.5, scale=0.5, size=len(col_1)), 
                 col_3 + np.random.normal(loc=0.5, scale=0.5, size=len(col_1))], axis=1)
sim_data = pd.DataFrame(data=data, columns=COL_NAMES[:data.shape[1]])

# lagged versions
true_data_lagged = ts_to_lagged(data=true_data, lagged_feats=None, lags=7, contemporaneous=True)
sim_data_lagged = ts_to_lagged(data=sim_data, lagged_feats=None, lags=7, contemporaneous=True)

# optimal simulation w/ bbc
res = get_optimal_sim_XY(
        true_data = true_data, 
        CONFIGS = None, 
        optimal_det_config = None,
        optimal_det_func = None, 
        sparsity_penalty=True,
        bias_correction=True,
        verbose = True
)

# results
print(res.keys())
print(f"opt: {res['auc']} | bbc: {res['bbc']}")